In [ ]:
#from pyhdf.SD import SD, SDC
import pandas as pd
import numpy as np
import dask.array as da
import dask.dataframe as dd
import time
import math
#import graphviz
from netCDF4 import Dataset
import os,datetime,sys,fnmatch
import h5py

# Reading functions for input MOD03 and MOD06_L2 Files
def read_MODIS_level2_data(MOD06_files,MOD03_files):
    print(MOD06_files)
    print(MOD03_files)
    print('reading the cloud mask from MOD06_L2 product')
    myd06 = Dataset(MOD06_files, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,:] # Reading Specific Variable 'Cloud_Mask_1km'.
    CM   = (np.array(CM[:,:,0],dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
    print('The Level-2 Cloud Mask Array Shape',CM.shape)
    print(' ')
    
    
    print('reading the lat-lon from MOD03 product')
    myd03 = Dataset(MOD03_files, "r")
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    longitude = np.array(longitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    print('The Level-2 Latitude-Longitude Array Shape',latitude.shape)
    print(' ')

    return latitude,longitude,CM

#Function for processing the cloud Fraction
def value_locate(refx, x):
    refx = np.array(refx)
    x = np.array(x)
    loc = np.zeros(len(x), dtype='int')
    
    for i in range(len(x)):
        ix = x[i]
        ind = ((refx - ix) <= 0).nonzero()[0]
        if len(ind) == 0:
            loc[i] = -1
        else: loc[i] = ind[-1]

    return loc

def division(n, d):

    div = np.zeros(len(d))
    for i in range(len(d)):
        if d[i] >0:
          div[i]=n[i]/d[i]
        else: div[i]=None 

    return div

def countzero(x, axis=1):
    #print(x)
    count0 = 0
    count1 = 0
    for i in x:
        if i <= 1:
            count0 +=1
    #print(count0/len(x))
    return count0/len(x)


MOD03_path = '/home/dprakas1/jianwu_common/MODIS_Aggregation/deepak-code/pandas_data/'
MOD06_path = '/home/dprakas1/jianwu_common/MODIS_Aggregation/deepak-code/pandas_data/'


satellite = 'Aqua'

yr = [2008]
mn = [1] #np.arange(1,13)  #[1]
dy = [1] #np.arange(1,32) # [1] #np.arange(1,31)
lat_bnd = np.arange(-90,91,1)# latitude and longtitude boundaries of level-3 grid
lon_bnd = np.arange(-180,180,1)
nlat = 180
nlon = 360

TOT_pix      = np.zeros(nlat*nlon)
CLD_pix      = np.zeros(nlat*nlon)


MOD03_fp = 'MYD03.A*.hdf'
MOD06_fp = 'MYD06_L2.A*.hdf'
MOD03_fn, MOD06_fn =[],[]
#MOD03_fn2, MOD06_fn2 =[],[]
MOD03_fn2, MOD06_fn2 =[],[]
for MOD06_flist in  os.listdir(MOD06_path):
    if fnmatch.fnmatch(MOD06_flist, MOD06_fp):
        MOD06_fn = MOD06_flist
        MOD06_fn2.append(MOD06_flist)
        #print(MOD06_fn)
for MOD03_flist in  os.listdir(MOD03_path):
    if fnmatch.fnmatch(MOD03_flist, MOD03_fp):
        MOD03_fn = MOD03_flist
        MOD03_fn2.append(MOD03_flist)
        #print(MOD03_fn)
if MOD03_fn and MOD06_fn:
    # if both MOD06 and MOD03 products are in the directory
    print('Reading Level 2 GeoLocation & Cloud Data')
    #print(MOD06_fn)
    #print(MOD03_fn)
    Lat,Lon,CM = read_MODIS_level2_data(MOD06_path+MOD06_fn,MOD03_path+MOD03_fn)
    
Lat.shape #Checking for the correctness of the latitude shape
Lon.shape #Checking for the correctness of the longitude shape
CM.shape #Checking for the correctness of the CM shape
    

cm = np.zeros((2030,1354), dtype=np.float32)
bit0 = np.zeros((2030,1354), dtype=np.float32)
bit12 = np.zeros((2030,1354), dtype=np.float32)    
myd06_name ='/home/dprakas1/jianwu_common/MODIS_Aggregation/deepak-code/pandas_data/'

for MOD06_file in MOD06_fn2:
    MOD06_file2 = myd06_name + MOD06_file
    #print(MOD06_file2)
    myd06 = Dataset(MOD06_file2, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,:]# Reading Specific Variable 'Cloud_Mask_1km'.
    CM = myd06.variables["Cloud_Mask_1km"][:,:,0]
    bit0r   = (np.array(CM,dtype='byte') & 0b00000001)
    bit12r   = (np.array(CM,dtype='byte') & 0b00000110) >>1
    #CM = np.array(CM).byteswap().newbyteorder()
    cm = np.concatenate((cm,CM))
    #bit0 = np.dstack((bit0,bit0r))
    #bit12 = np.dstack((bit12,bit12r))
    
print('The Cloud Mask Array Shape Is: ',cm.shape)

    
myd03_name = '/home/dprakas1/jianwu_common/MODIS_Aggregation/deepak-code/pandas_data/'
lat = np.zeros((2030,1354), dtype=np.float32)
lon = np.zeros((2030,1354), dtype=np.float32)
for MOD03_file in MOD03_fn2:
    MOD03_file2 = myd03_name + MOD03_file
    
    myd03 = Dataset(MOD03_file2, "r")
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    lat = np.concatenate((lat,latitude))


    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    lon = np.concatenate((lon,longitude))
    
print('Longitude Shape Is: ',lon.shape)
print('Latitude Shape Is: ',lat.shape)

cm=np.ravel(cm)
lat=np.ravel(lat)
lon=np.ravel(lon)

print("printing")
cm.shape
lat.shape
lon.shape

print("printing")

d = {'Latitude' :  pd.Series(lat), 'Longitude' : pd.Series(lon),'CM':pd.Series(cm)} #Reading the values in a pandas dataframe
df = pd.DataFrame(d,columns=['Latitude', 'Longitude','CM']) #Creating a pandas dataframe
df=df.astype(np.int8)
df2=df.groupby(['Longitude','Latitude'])
df2.CM.apply(countzero)
df3=df2.CM.apply(countzero)
df3=dd.concat([df3],axis=1)

print("printing")


#to convert the dataframe to hdf file
total_cloud_fraction = df3
out_name = 'pandas_onefile.hdf5'
def save_hdf(out_name,total_cloud_fraction,lat_bnd,lon_bnd):
    f=h5py.File(out_name,'w')
    PCentry=f.create_dataset('CF',data=total_cloud_fraction)
    PCentry.dims[0].label='lat_bnd'
    PCentry.dims[1].label='lon_bnd'

    PC=f.create_dataset('lat_bnd',data=lat_bnd)
    PC.attrs['units']='degrees'
    PC.attrs['long_name']='Latitude_boundaries'

    PC=f.create_dataset('lon_bnd',data=lon_bnd)
    PC.attrs['units']='degrees'
    PC.attrs['long_name']='Longitude_boundaries'
    f.close()
    print(out_name+' Saved!!')

save_hdf(out_name,total_cloud_fraction,lat_bnd,lon_bnd)